In [2]:
%pip install -r requirements.txt

  Created wheel for dj-database-url: filename=dj_database_url-0.4.1-py2.py3-none-any.whl size=3785 sha256=858b798298e2cc5ec5f2c2bc64aa21bca60626032ab895226dbbd891c496e39e
  Stored in directory: c:\users\b7e3~1\appdata\local\pip\cache\wheels\e9\0f\ff\be6ffd2dfe3f7e77ab41ccf5558f8455fdc4bbfa3c106eee71
  Created wheel for telepot: filename=telepot-12.7-py3-none-any.whl size=57967 sha256=c31dece9bd68d7e4df98d330b9f0b5ffe5bcab1fedd52ea605e3bb05ab2b32d2
  Stored in directory: c:\users\b7e3~1\appdata\local\pip\cache\wheels\59\4d\ea\8781198a69408b6c37d47d902d7bbe5541969eddf5290e6b2e
  Created wheel for telebot: filename=telebot-0.0.3-py3-none-any.whl size=3791 sha256=4ef11ccbbd49c9495ca7cda6ab2bb8e22c90cea8be8632926ffb6a66e23d1928
  Stored in directory: c:\users\b7e3~1\appdata\local\pip\cache\wheels\d0\c3\8d\c94c14e1ed91e35996a523e4856eea50f1457ad96f021e7c29
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.2-py3-none-any.whl size=52968 sha256=32751decb8b7e8858cb7736a513f62a7

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spacy 2.3.2 requires requests<3.0.0,>=2.13.0, but you'll have requests 2.9.1 which is incompatible.
google-api-core 1.22.0 requires requests<3.0.0dev,>=2.18.0, but you'll have requests 2.9.1 which is incompatible.
conda 4.8.4 requires requests>=2.12.4, but you'll have requests 2.9.1 which is incompatible.
telebot 0.0.3 requires requests==2.7.0, but you'll have requests 2.9.1 which is incompatible.


In [1]:
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import pickle

from flask import Flask
from flask import request
import requests
from flask import jsonify

import os
import json
from ast import literal_eval
import traceback

application = Flask(__name__)


#загружаем модели из файла
vec = pickle.load(open("./models/tfidf.pickle", "rb"))
model = lgb.Booster(model_file='./models/lgbm_model.txt')


# тестовый вывод
@application.route("/")  
def hello():
    resp = {'message':"Hello World!"}
    
    response = jsonify(resp)
    
    return response

# предикт категории
#{"user_message":"example123rfssg gsfgfd"}
@application.route("/categoryPrediction", methods=['GET', 'POST'])  
def registration():
    resp = {'message':'ok'
           ,'category': -1
           }

    try:
        getData = request.get_data()
        json_params = json.loads(getData) 
        
        #напишите прогноз и верните его в ответе в параметре 'prediction'
        

        
    except Exception as e: 
        print(e)
        resp['message'] = e
      
    response = jsonify(resp)
    
    return response

        

if __name__ == "__main__":
    port = int(os.getenv('PORT', 5000))
    application.run(debug=False, port=port, host='0.0.0.0' , threaded=True)





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\Public\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Public\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


# Пример сохранения моделей и прогноза

In [ ]:
#сохраняем векторайзер
import pickle 

labels, levels = pd.factorize(df['category'])
df['category_int'] = labels

texts = df['message']

vec = TfidfVectorizer(max_features=100)
tfidf_mat = vec.fit_transform(texts).toarray()


#vec - модель из прошлой практики

#сохраняем модель в файл
pickle.dump(vec, open("tfidf.pickle", "wb"))

#загружаем модель из файла
vec = pickle.load(open("tfidf.pickle", "rb"))



#### сохраняем бустинг
import lightgbm as lgb
train = pd.DataFrame(tfidf_mat)
train['category_int'] = df['category_int']

train.dropna(inplace = True)

target = 'category_int'
train_columns = list(set(train.columns) - set([target]))


lgb_data_train = lgb.Dataset(train[train_columns], 
                             train[target].values, 
                             free_raw_data=False
                            )
params = {
    'objective':'multiclass', 
    'metric': 'multi_logloss',
    "num_class" : 3,
    'learning_rate': 0.01, 
    'random_state':202008,
    'subsample':0.33,
    'colsample_bytree':0.33,
    'reg_lambda':4
}

model = lgb.train(params, lgb_data_train, num_boost_round=100  )

model.save_model('lgbm_model.txt')

model = lgb.Booster(model_file='lgbm_model.txt')

#test predict
message = 'hello i want to payment but it is blocked'
model.predict(vec.transform([message]).toarray())